In [592]:
import copy
from collections import deque
import queue

In [593]:
class node():
    def __init__(self, nodeLevel=None, listOfJugs = [], parentNode = None):
        self.nodeLevel = nodeLevel
        self.listOfJugs = listOfJugs #last jug has Infinite capacity  
        self.g = self.nodeLevel
        self.h = 0
        self.parentNode = parentNode
        
        self.jugVolumes = [jug.volume for jug in self.listOfJugs]
        self.nodeID = str(self.nodeLevel) + str(self.jugVolumes) #Unique ID of the Node
        
    def computeF(self):
        self.f = self.g + self.h
        
        return self.f
    

In [784]:
def heuristic(node, target):
    
    sumOfVolumes = 0
    maxVolume = 0
    minCapacity = 2**100
    for jug in node.listOfJugs[:-1]:
        if jug.volume > maxVolume:
            maxVolume = jug.volume
        if jug.capacity < minCapacity:
            minCapacity = jug.capacity
            
        sumOfVolumes += jug.volume
        
    return abs(int(target - node.listOfJugs[-1].volume - maxVolume))
    #return abs(int(target - node.listOfJugs[-1].volume - maxVolume) / minCapacity) * 2
    #return abs(int(((target - sumOfVolumes - node.listOfJugs[-1].volume) / minCapacity) * 2 ))

In [595]:
def createNodes(sourceNode):
    nodes = []
    
    #EMPTY JUGs
    for i in range(0,len(sourceNode.listOfJugs)-1):
        tempJugs = copy.deepcopy(sourceNode.listOfJugs)
        
        if tempJugs[i].isEmpty():
            continue 
        
        else:
            tempJugs[i].pour(tempJugs[i].volume)
            newNode = node(sourceNode.nodeLevel + 1, tempJugs, parentNode = sourceNode)
            nodes.append(newNode)

    #FILL JUGs
    for i in range(0,len(sourceNode.listOfJugs)-1):
        tempJugs = copy.deepcopy(sourceNode.listOfJugs)
        
        if tempJugs[i].isFull():
            continue
            
        else:
            tempJugs[i].fill()
            newNode = node(sourceNode.nodeLevel + 1, tempJugs, parentNode = sourceNode)
            nodes.append(newNode)

    #pour from ith jug to INF jug
    for i in range(0,len(sourceNode.listOfJugs)-1):
        tempJugs = copy.deepcopy(sourceNode.listOfJugs)
        
        if tempJugs[i].isEmpty():
            continue
        else:
            pourWater(tempJugs[i], tempJugs[-1])
            newNode = node(sourceNode.nodeLevel + 1, tempJugs, parentNode = sourceNode)
            nodes.append(newNode)
    
    #pour from INF jug to ith jug
    for i in range(0,len(sourceNode.listOfJugs)-1):
        tempJugs = copy.deepcopy(sourceNode.listOfJugs)
        
        if tempJugs[-1].isEmpty():
            continue
        else:
            pourWater(tempJugs[-1], tempJugs[i])
            newNode = node(sourceNode.nodeLevel + 1, tempJugs, parentNode = sourceNode)
            nodes.append(newNode)
            
    #pour water from one jug to another for every combination 
    for i in range(0,len(sourceNode.listOfJugs)-2):
        for j in range(i+1, len(sourceNode.listOfJugs)-1):
            tempJugs1 = copy.deepcopy(sourceNode.listOfJugs)
            tempJugs2 = copy.deepcopy(sourceNode.listOfJugs)
            
            #pour from ith jug to jth jug
            if tempJugs1[j].isFull() or tempJugs1[i].isEmpty():
                continue
            else:
                pourWater(tempJugs1[i], tempJugs1[j])
                newNode = node(sourceNode.nodeLevel + 1, tempJugs1, parentNode = sourceNode)
                nodes.append(newNode)
                
            #pour from jth jug to ith jug
            if tempJugs2[i].isFull() or tempJugs2[j].isEmpty():
                continue
            else:
                pourWater(tempJugs2[j], tempJugs2[i])
                newNode = node(sourceNode.nodeLevel + 1, tempJugs2, parentNode = sourceNode)
                nodes.append(newNode)
            
    return nodes

In [596]:
class jug():
    def __init__(self, capacity, volume = 0):
        self.capacity = capacity
        self.volume = volume
    
    def fill(self, volume = None):
        if volume is None:
            self.volume = self.capacity
        else:
            self.volume = volume
            
    def isEmpty(self):
        if self.volume == 0:
            return True
        else:
            return False
    
    def isFull(self):
        if self.volume == self.capacity:
            return True
        else:
            return False
        
    def pour(self,volume = None):
        if volume == None:
            temp = self.volume
            self.volume = 0
        
            return temp 
        else:
            if volume > self.volume:
                temp = self.volume
                self.volume = 0
                return temp
            else: 
                self.volume = self.volume - volume
                return volume
                 
    def availableCapacity(self):
        return self.capacity - self.volume

In [597]:
def pourWater(jugA, jugB):
        
        if jugA.volume < jugB.availableCapacity():
            jugB.volume = jugB.volume + jugA.pour()
        
        else:
            jugB.volume = jugB.volume + jugA.pour(jugB.availableCapacity())
            

In [598]:
def goalTest(tempNode, target):
    if tempNode.listOfJugs[-1].volume == target:
        return True
    else:
        return False
    

In [599]:
def printNode(node):
    print("NODE LEVEL: " + str(node.nodeLevel))
    for jug in node.listOfJugs:
        print(jug.volume)

In [600]:
def printPath(node):
    while node.parentNode is not None:
        printNode(node)
        node = node.parentNode

In [774]:
def mainTest(capacities, target):
    
    jugs = [jug(int(capacity)) for capacity in capacities]
    jugInf = jug(2**100)
    jugs.append(jugInf)
    nodeInit = node(0, jugs) 
    nodeInit.h = heuristic(nodeInit,target)
    
    result = aStar(nodeInit, target)
    if result == -1:
        return -1
    else:
        return result.nodeLevel

In [775]:
def test():
    assert mainTest([1,4,10,15,22], 181) == 20, "Should be 19"
    assert mainTest([2,5,6,72], 143) == 10, "Should be 10"
    assert mainTest([3,6], 2) == -1, "Should be -1"
    assert mainTest([2], 143) == -1, "Should be -1"
    assert mainTest([2,3,5,19,121,852], 11443) == 36, "Should be 36"

In [778]:
test()

THERE IS NO PATH AVAILABLE
THERE IS NO PATH AVAILABLE


In [790]:
def aStar(initialNode, target):
    nodeInit = initialNode

    frontier = queue.PriorityQueue()
    frontier.put((nodeInit.computeF(), 0, nodeInit))

    explored = set()
    frontierSet = dict()

    entryCounter = 1
    while( frontier.qsize() > 0):

        (currNodePriority, _ , currNode) = frontier.get()

        printNode(currNode)
        print("CURR NODE F VAL : " + str(currNode.f))
        
        if currNode.f > target * 10:
            break
        
        #TEST The node after popping and item from priority queue
        if goalTest(currNode, target):
            goalNode = currNode
            #print(goalNode.nodeLevel)
            #printNode(currNode)
            
            return goalNode

        explored.add(str(currNode.jugVolumes))

        createdNodes = createNodes(currNode)

        for tempNode in createdNodes:
            tempNode.h = heuristic(tempNode,target)
            tempNode.computeF()


            if str(tempNode.jugVolumes) not in explored and str(tempNode.jugVolumes) not in frontierSet:
                frontier.put((tempNode.f, entryCounter, tempNode))
                frontierSet[str(tempNode.jugVolumes)] = tempNode.f
                entryCounter +=1
                 #entryCounter is kept to have a priority over the nodes that has same f value
                
            elif str(tempNode.jugVolumes) in frontierSet:
                #if the node is also reachable through a shorter path 
                if tempNode.f < frontierSet[str(tempNode.jugVolumes)]: 
                    frontier.put((tempNode.f, entryCounter, tempNode))
                    frontierSet[str(tempNode.jugVolumes)] = tempNode.f
                    entryCounter +=1
    
    print("THERE IS NO PATH AVAILABLE")
    return -1
                   

In [791]:
file = open("input.txt", "r")
capacities = file.readline().rstrip('\n')
capacities = capacities.split(",")
target = int(file.readline().rstrip('\n'))

jugs = [jug(int(capacity)) for capacity in capacities]

jugInf = jug(2**100)

jugs.append(jugInf)

nodeInit = node(0, jugs) 
nodeInit.h = heuristic(nodeInit,target)

In [792]:
goalNode = aStar(nodeInit, target)

NODE LEVEL: 0
0
0
CURR NODE F VAL : 143
NODE LEVEL: 1
2
0
CURR NODE F VAL : 142
NODE LEVEL: 2
0
2
CURR NODE F VAL : 143
NODE LEVEL: 3
2
2
CURR NODE F VAL : 142
NODE LEVEL: 4
0
4
CURR NODE F VAL : 143
NODE LEVEL: 5
2
4
CURR NODE F VAL : 142
NODE LEVEL: 6
0
6
CURR NODE F VAL : 143
NODE LEVEL: 7
2
6
CURR NODE F VAL : 142
NODE LEVEL: 8
0
8
CURR NODE F VAL : 143
NODE LEVEL: 9
2
8
CURR NODE F VAL : 142
NODE LEVEL: 10
0
10
CURR NODE F VAL : 143
NODE LEVEL: 11
2
10
CURR NODE F VAL : 142
NODE LEVEL: 12
0
12
CURR NODE F VAL : 143
NODE LEVEL: 13
2
12
CURR NODE F VAL : 142
NODE LEVEL: 14
0
14
CURR NODE F VAL : 143
NODE LEVEL: 15
2
14
CURR NODE F VAL : 142
NODE LEVEL: 16
0
16
CURR NODE F VAL : 143
NODE LEVEL: 17
2
16
CURR NODE F VAL : 142
NODE LEVEL: 18
0
18
CURR NODE F VAL : 143
NODE LEVEL: 19
2
18
CURR NODE F VAL : 142
NODE LEVEL: 20
0
20
CURR NODE F VAL : 143
NODE LEVEL: 21
2
20
CURR NODE F VAL : 142
NODE LEVEL: 22
0
22
CURR NODE F VAL : 143
NODE LEVEL: 23
2
22
CURR NODE F VAL : 142
NODE LEVEL: 

CURR NODE F VAL : 392
NODE LEVEL: 268
0
268
CURR NODE F VAL : 393
NODE LEVEL: 269
2
268
CURR NODE F VAL : 396
NODE LEVEL: 270
0
270
CURR NODE F VAL : 397
NODE LEVEL: 271
2
270
CURR NODE F VAL : 400
NODE LEVEL: 272
0
272
CURR NODE F VAL : 401
NODE LEVEL: 273
2
272
CURR NODE F VAL : 404
NODE LEVEL: 274
0
274
CURR NODE F VAL : 405
NODE LEVEL: 275
2
274
CURR NODE F VAL : 408
NODE LEVEL: 276
0
276
CURR NODE F VAL : 409
NODE LEVEL: 277
2
276
CURR NODE F VAL : 412
NODE LEVEL: 278
0
278
CURR NODE F VAL : 413
NODE LEVEL: 279
2
278
CURR NODE F VAL : 416
NODE LEVEL: 280
0
280
CURR NODE F VAL : 417
NODE LEVEL: 281
2
280
CURR NODE F VAL : 420
NODE LEVEL: 282
0
282
CURR NODE F VAL : 421
NODE LEVEL: 283
2
282
CURR NODE F VAL : 424
NODE LEVEL: 284
0
284
CURR NODE F VAL : 425
NODE LEVEL: 285
2
284
CURR NODE F VAL : 428
NODE LEVEL: 286
0
286
CURR NODE F VAL : 429
NODE LEVEL: 287
2
286
CURR NODE F VAL : 432
NODE LEVEL: 288
0
288
CURR NODE F VAL : 433
NODE LEVEL: 289
2
288
CURR NODE F VAL : 436
NODE LEVEL

NODE LEVEL: 494
0
494
CURR NODE F VAL : 845
NODE LEVEL: 495
2
494
CURR NODE F VAL : 848
NODE LEVEL: 496
0
496
CURR NODE F VAL : 849
NODE LEVEL: 497
2
496
CURR NODE F VAL : 852
NODE LEVEL: 498
0
498
CURR NODE F VAL : 853
NODE LEVEL: 499
2
498
CURR NODE F VAL : 856
NODE LEVEL: 500
0
500
CURR NODE F VAL : 857
NODE LEVEL: 501
2
500
CURR NODE F VAL : 860
NODE LEVEL: 502
0
502
CURR NODE F VAL : 861
NODE LEVEL: 503
2
502
CURR NODE F VAL : 864
NODE LEVEL: 504
0
504
CURR NODE F VAL : 865
NODE LEVEL: 505
2
504
CURR NODE F VAL : 868
NODE LEVEL: 506
0
506
CURR NODE F VAL : 869
NODE LEVEL: 507
2
506
CURR NODE F VAL : 872
NODE LEVEL: 508
0
508
CURR NODE F VAL : 873
NODE LEVEL: 509
2
508
CURR NODE F VAL : 876
NODE LEVEL: 510
0
510
CURR NODE F VAL : 877
NODE LEVEL: 511
2
510
CURR NODE F VAL : 880
NODE LEVEL: 512
0
512
CURR NODE F VAL : 881
NODE LEVEL: 513
2
512
CURR NODE F VAL : 884
NODE LEVEL: 514
0
514
CURR NODE F VAL : 885
NODE LEVEL: 515
2
514
CURR NODE F VAL : 888
NODE LEVEL: 516
0
516
CURR NODE 

712
CURR NODE F VAL : 1281
NODE LEVEL: 713
2
712
CURR NODE F VAL : 1284
NODE LEVEL: 714
0
714
CURR NODE F VAL : 1285
NODE LEVEL: 715
2
714
CURR NODE F VAL : 1288
NODE LEVEL: 716
0
716
CURR NODE F VAL : 1289
NODE LEVEL: 717
2
716
CURR NODE F VAL : 1292
NODE LEVEL: 718
0
718
CURR NODE F VAL : 1293
NODE LEVEL: 719
2
718
CURR NODE F VAL : 1296
NODE LEVEL: 720
0
720
CURR NODE F VAL : 1297
NODE LEVEL: 721
2
720
CURR NODE F VAL : 1300
NODE LEVEL: 722
0
722
CURR NODE F VAL : 1301
NODE LEVEL: 723
2
722
CURR NODE F VAL : 1304
NODE LEVEL: 724
0
724
CURR NODE F VAL : 1305
NODE LEVEL: 725
2
724
CURR NODE F VAL : 1308
NODE LEVEL: 726
0
726
CURR NODE F VAL : 1309
NODE LEVEL: 727
2
726
CURR NODE F VAL : 1312
NODE LEVEL: 728
0
728
CURR NODE F VAL : 1313
NODE LEVEL: 729
2
728
CURR NODE F VAL : 1316
NODE LEVEL: 730
0
730
CURR NODE F VAL : 1317
NODE LEVEL: 731
2
730
CURR NODE F VAL : 1320
NODE LEVEL: 732
0
732
CURR NODE F VAL : 1321
NODE LEVEL: 733
2
732
CURR NODE F VAL : 1324
NODE LEVEL: 734
0
734
CURR N

In [769]:
printPath(goalNode)

NODE LEVEL: 10
0
0
0
0
143
NODE LEVEL: 9
0
0
0
72
71
NODE LEVEL: 8
0
0
6
72
71
NODE LEVEL: 7
2
0
6
72
71
NODE LEVEL: 6
2
0
5
72
72
NODE LEVEL: 5
2
5
0
72
72
NODE LEVEL: 4
2
0
0
72
72
NODE LEVEL: 3
2
0
0
0
72
NODE LEVEL: 2
2
0
0
72
0
NODE LEVEL: 1
0
0
0
72
0
